<a href="https://colab.research.google.com/github/arva29/AMDproject/blob/main/UkraineWarTweetsProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# this is meant to be run on google colab

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init("spark-2.4.5-bin-hadoop2.7") #SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 70 bytes


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
!kaggle datasets download -d bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows

100% 12.0G/12.0G [01:16<00:00, 200MB/s]
100% 12.0G/12.0G [01:16<00:00, 169MB/s]


In [6]:
!unzip ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip

Archive:  ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip
  inflating: 0819_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0820_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0821_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0822_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0823_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0824_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0825_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0826_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0827_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0828_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0829_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0830_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0831_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0901_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0902_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0903_UkraineCombinedTweetsDeduped.csv.gzip  
  inflat

I extract the collection of tweets from the 21 of September and I will work on this.

In [18]:
import csv
import gzip

data_list = []

with gzip.open('0921_UkraineCombinedTweetsDeduped.csv.gzip', 'rt') as f:
    reader = csv.DictReader(f)
    for row in reader:
        data_list.append(row)

rdd = sc.parallelize(data_list)

In [19]:
rdd.count()

104313

In [20]:
rdd.first()

OrderedDict([('', '71762'),
             ('userid', '1268144739997503493'),
             ('username', 'DailyBeijing'),
             ('acctdesc', 'Official Twitter Account of The Beijing Daily.'),
             ('location', 'Beijing, China'),
             ('following', '242'),
             ('followers', '2113'),
             ('totaltweets', '10142'),
             ('usercreatedts', '2020-06-03 11:49:43'),
             ('tweetid', '1572375052942573568'),
             ('tweetcreatedts', '2022-09-21 00:00:00'),
             ('retweetcount', '0'),
             ('text',
              '.#Britain’s new Prime Minister Liz Truss will pledge to match or exceed £2.3 billion ($2.63 billion) of #military aid to #Ukraine for the year 2023, a government statement said Tuesday. https://t.co/9dI1nmWjzn'),
             ('hashtags',
              "[{'text': 'Britain', 'indices': [1, 9]}, {'text': 'military', 'indices': [104, 113]}, {'text': 'Ukraine', 'indices': [121, 129]}]"),
             ('language', 'en

In [34]:
rdd.filter(lambda x: x.get("language")=='en').map(lambda x: x.get("text")).take(40)

['.#Britain’s new Prime Minister Liz Truss will pledge to match or exceed £2.3 billion ($2.63 billion) of #military aid to #Ukraine for the year 2023, a government statement said Tuesday. https://t.co/9dI1nmWjzn',
 'EXTRA INCOME – by Environmental Protection with INTEREST RATE\n\nCLICK &gt; https://t.co/DmfNL6wQSN\n\n#dollar #extra #income #dream #euro #usa #germany #europe #china #president #moscow #ukraine  #uk #bitcoin #btc #desire #environment #Renewable #solar #hydro #wind #pam #palm #climate https://t.co/oCx1w1CT0B',
 '.#Britain’s new Prime Minister Liz Truss will pledge to match or exceed £2.3 billion ($2.63 billion) of #military aid to #Ukraine for the year 2023, a government statement said Tuesday. https://t.co/h2tWRdGVQA',
 "'War with potential adversaries can never be ruled out...': Navy Chief weighs in on 'challenges' posed by #China #Pakistan \n\nhttps://t.co/8YZ4gJFAa9",
 'I hope this is true. 🤞 ❤️ 🇺🇦 #SlavaUkraini \n\n🖕🏻🇷🇺 #PuckFutin https://t.co/5oWlVtwbz2',
 'Rothschil